In [20]:
%%time
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 47.2 µs


In [21]:
%%time
# Generate the SMIRNOFF ligand Structure object
ifs = oechem.oemolistream('../OpenMMCubes/tests/input/toluene.pdb')
mol = oechem.OEMol()
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
oechem.OEReadMolecule(ifs, mol )
oechem.OETriposAtomNames(mol)

from smarty.forcefield import ForceField
mol_ff = ForceField('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml')
mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
molecule_structure = parmed.openmm.load_topology(mol_top, mol_sys, xyz=mol_pos) 
molecule_structure.residues[0].name = "MOL"

print(mol_ff)
print(molecule_structure)
print(molecule_structure.topology)

<Structure 15 atoms; 1 residues; 15 bonds; parametrized>
<Topology; 1 chains, 1 residues, 15 atoms, 15 bonds>
CPU times: user 208 ms, sys: 0 ns, total: 208 ms
Wall time: 207 ms


In [22]:
%%time
#Generate protein Stucture object
pdbfilename = os.path.join('..','OpenMMCubes', 'tests', 'input', 'T4-protein.pdb')
pdbfile = app.PDBFile(pdbfilename)
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')
system = forcefield.createSystem( pdbfile.topology )
protein_structure = parmed.openmm.load_topology( pdbfile.topology, system)

print(protein_structure)
print(protein_structure.topology)

<Structure 2634 atoms; 164 residues; 2654 bonds; parametrized>
<Topology; 1 chains, 164 residues, 2634 atoms, 2654 bonds>
CPU times: user 2.32 s, sys: 8 ms, total: 2.33 s
Wall time: 2.34 s


In [23]:
%%time
# Merge structures
structure = protein_structure + molecule_structure
topology = structure.topology

# Concatenate positions arrays
positions_unit = unit.angstroms
positions0_dimensionless = np.array( pdbfile.positions / positions_unit )
positions1_dimensionless = np.array( mol_pos / positions_unit )

coordinates = np.vstack((positions0_dimensionless,positions1_dimensionless))
natoms = len(coordinates)
positions = np.zeros([natoms,3], np.float32)
for index in range(natoms):
    (x,y,z) = coordinates[index]
    positions[index,0] = x
    positions[index,1] = y
    positions[index,2] = z
positions = unit.Quantity(positions, positions_unit)

# Store in Structure object
structure.coordinates = coordinates

# Save to PDB
structure.save('tmp.pdb',overwrite=True)

CPU times: user 988 ms, sys: 12 ms, total: 1e+03 ms
Wall time: 995 ms


In [24]:
%%time
# Solvate with PDBFixer
fixer = pdbfixer.PDBFixer('tmp.pdb')
#fixer.findMissingResidues()
#fixer.findMissingAtoms()
#fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=unit.Quantity(10, unit.angstroms),
                 ionicStrength=unit.Quantity(50, unit.millimolar))

CPU times: user 1min 46s, sys: 28 ms, total: 1min 46s
Wall time: 31.7 s


In [25]:
%%time
# Load PDBFixer object back to Structure
struct = parmed.openmm.load_topology(fixer.topology, xyz=fixer.positions)
# Remove from ligand from protein Structure
struct.strip(":MOL")
struct

CPU times: user 2.02 s, sys: 8 ms, total: 2.02 s
Wall time: 2.02 s


In [26]:
%%time
# Regenerate OpenMM System to parameterize solvent
system = forcefield.createSystem(struct.topology, rigidWater=False)

CPU times: user 3.71 s, sys: 12 ms, total: 3.72 s
Wall time: 3.72 s


In [27]:
%%time
# Regenerate parameterized protein structure
protein_structure = parmed.openmm.load_topology( struct.topology, system=system, 
                                                xyz=struct.positions, box=struct.box )

CPU times: user 6.14 s, sys: 28 ms, total: 6.17 s
Wall time: 6.17 s


In [28]:
%%time
# Remerge with ligand structure
combined_structure = protein_structure + molecule_structure

# Restore initial positions and box dimensions
combined_structure.positions = fixer.positions
combined_structure.box = struct.box
os.remove('tmp.pdb')
combined_structure.save('combined_structure.pdb', overwrite=True)

CPU times: user 3.41 s, sys: 44 ms, total: 3.46 s
Wall time: 3.45 s


In [29]:
%%time
# Regenerate OpenMM system with ParmEd
system = combined_structure.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=10.0*unit.angstroms), 
                                #constraints=app.HBonds)

CPU times: user 1.96 s, sys: 0 ns, total: 1.96 s
Wall time: 1.96 s


In [30]:
%%time
pdbfilename = 'combined_structure.pdb'
pdbfile = app.PDBFile(pdbfilename)
topology = pdbfile.topology
positions = pdbfile.positions

CPU times: user 3.21 s, sys: 40 ms, total: 3.25 s
Wall time: 3.25 s


In [31]:
%%time
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 207 µs


In [35]:
simulation = app.Simulation(topology, system, integrator )

NotImplementedError: Wrong number or type of arguments for overloaded function 'new_Context'.
  Possible C/C++ prototypes are:
    OpenMM::Context::Context(OpenMM::System const &,OpenMM::Integrator &)
    OpenMM::Context::Context(OpenMM::System const &,OpenMM::Integrator &,OpenMM::Platform &)
    OpenMM::Context::Context(OpenMM::System const &,OpenMM::Integrator &,OpenMM::Platform &,std::map< std::string,std::string,std::less< std::string >,std::allocator< std::pair< std::string const,std::string > > > const &)
    OpenMM::Context::Context(OpenMM::Context const &)


In [34]:
%%time
simulation.context.setPositions(positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

Exception: Called setPositions() on a Context with the wrong number of positions

In [15]:
print(simulation.context.getPlatform().getName())

CPU


In [ ]:
%%time
simulation.minimizeEnergy(maxIterations=100)

In [ ]:
%%time
statereporter = app.StateDataReporter('output.log', 100, step=True, potentialEnergy=True, temperature=True)
simulation.reporters.append(statereporter)

In [ ]:
%%time
simulation.step(1000)

In [ ]:
%%time
state = simulation.context.getState( getPositions=True,
                                     getVelocities=True,
                                     getForces=True,
                                     getEnergy=True,
                                     getParameters=True,
                                     enforcePeriodicBox=True )

In [46]:
pdbfile = open('../OpenMMCubes/tests/input/toluene.pdb', 'rb')
with oechem.oemolostream('smirff_mol.oeb.gz') as ofs:
    ffxml = open('../OpenMMCubes/tests/input/forcefield/smirff99Frosst.ffxml', 'rb')
    mol.SetData(oechem.OEGetTag('forcefield'), ffxml.read())
    mol.SetData(oechem.OEGetTag('pdb'), pdbfile.read())
    oechem.OEWriteMolecule(ofs, mol)

In [65]:
import io
pdb = mol.GetData(oechem.OEGetTag('pdb'))
output = io.StringIO()
output.write(pdb)
output.seek(0)
print(type(output))

pdbfile = app.PDBFile(output)
print(pdbfile.positions)


<class '_io.StringIO'>
[(2.237, 2.168, 2.9770000000000003), (2.2600000000000002, 2.293, 3.0420000000000003), (2.34, 2.108, 2.9010000000000002), (2.3760000000000003, 2.359, 3.0250000000000004), (2.462, 2.176, 2.895), (2.483, 2.298, 2.958), (2.6100000000000003, 2.368, 2.94), (2.1390000000000002, 2.122, 2.986), (2.183, 2.342, 3.1020000000000003), (2.315, 2.0180000000000002, 2.846), (2.399, 2.4570000000000003, 3.068), (2.5440000000000005, 2.1270000000000002, 2.841), (2.591, 2.4730000000000003, 2.915), (2.6670000000000003, 2.3640000000000003, 3.0340000000000003), (2.6790000000000003, 2.3190000000000004, 2.872)] nm


In [4]:
# Generate the SMIRNOFF ligand Structure object
with oechem.oemolistream('smirff_mol.oeb.gz') as ifs:
    mol = oechem.OEMol()
    oechem.OEReadMolecule(ifs, mol)
    ffxml = mol.GetData(oechem.OEGetTag('forcefield')).encode()
    
    with open('smirff.ffxml', 'wb') as out:
        out.write(ffxml)
        
    from smarty.forcefield import ForceField
    mol_ff = ForceField(open('smirff.ffxml'))
    mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
    print(mol_pos)

[[ 22.37000084  21.68000031  29.77000046]
 [ 22.60000038  22.93000031  30.42000008]
 [ 23.39999962  21.07999992  29.01000023]
 [ 23.76000023  23.59000015  30.25      ]
 [ 24.62000084  21.76000023  28.95000076]
 [ 24.82999992  22.97999954  29.57999992]
 [ 26.10000038  23.68000031  29.39999962]
 [ 21.38999939  21.21999931  29.86000061]
 [ 21.82999992  23.42000008  31.02000046]
 [ 23.14999962  20.18000031  28.45999908]
 [ 23.98999977  24.56999969  30.68000031]
 [ 25.44000053  21.27000046  28.40999985]
 [ 25.90999985  24.72999954  29.14999962]
 [ 26.67000008  23.63999939  30.34000015]
 [ 26.79000092  23.19000053  28.71999931]] A


In [105]:
oebfname = '../simulation.oeb.gz'
systag = oechem.OEGetTag('states')
ifs = oechem.oemolistream(oebfname)
for mol in ifs.GetOEMols():
    data = mol.GetData(systag)
    system = openmm.XmlSerializer.deserialize( data )
    print(type(system))

ValueError: GetData: states not found.

In [92]:
systag = oechem.OEGetTag('system')
sys_check = oechem.OEHasTag('system')
serialized_system =  mol.GetData(systag)
system = openmm.XmlSerializer.deserialize( serialized_system )
print(type(system))

<class 'simtk.openmm.openmm.System'>


In [46]:
oebfname = '../simulation.oeb.gz'
systag = oechem.OEGetTag('system')
statetag = oechem.OEGetTag('state')
logtag = oechem.OEGetTag('log')
ifs = oechem.oemolistream(oebfname)
for mol in ifs.GetOEMols():
    print(mol)
    system = openmm.XmlSerializer.deserialize( mol.GetData(systag) )
    print(type(system))
    
    state = openmm.XmlSerializer.deserialize( mol.GetData(statetag) )
    print(type(state))
    
    log = mol.GetData(logtag)
    print(log)
    

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7fa1a64e3f00> >
<class 'simtk.openmm.openmm.System'>
<class 'simtk.openmm.openmm.State'>
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-428542.16782426834,268.27374506936474
2000,-415328.90217246115,292.01381518311763
3000,-410227.29526019096,300.52425729114134
4000,-410178.7364115864,296.1576736337405
5000,-408896.961594522,300.5506400232759
6000,-409723.38068178296,299.9514962746391
7000,-410476.6667842157,299.47896456019174
8000,-407904.6782325804,300.0447198094963
9000,-409309.79629419,299.9292946386496
10000,-409230.8660710901,301.3697347440811

